# Part 3: LLM 기반 엔티티/관계 자동 추출

**소요시간:** 2시간  
**난이도:** ★★★☆  
**마일스톤:** 자동 추출 KG + 품질 리포트 — 수작업 vs LLM 비교표

---

## 학습 목표

1. LLM 기반 엔티티/관계 추출을 위한 **프롬프트 설계** 방법론 이해
2. 기본 / 스키마 주입 / Few-shot 프롬프트의 **품질 차이** 체험
3. OpenAI GPT-4o를 활용한 **배치 추출** 파이프라인 구축
4. 수작업 vs LLM 추출 결과의 **정량적 비교**
5. Neo4j에 추출 결과 **자동 적재**

---
## 1. 환경 설정

OpenAI API와 Neo4j 데이터베이스에 연결하고, 실습용 뉴스 데이터를 로드합니다.

In [ ]:
# 필수 패키지 임포트
import json
import os
import time
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from openai import OpenAI
from neo4j import GraphDatabase
from dotenv import load_dotenv

# 한글 폰트 설정 (matplotlib)
matplotlib.rcParams['font.family'] = 'AppleGothic'  # macOS
# matplotlib.rcParams['font.family'] = 'Malgun Gothic'  # Windows
matplotlib.rcParams['axes.unicode_minus'] = False

print("패키지 로드 완료")

In [ ]:
# 환경변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Neo4j 연결
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "graphrag2024")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# 연결 테스트
with driver.session() as session:
    result = session.run("RETURN 1 AS test")
    print(f"Neo4j 연결 성공: {result.single()['test']}")

print(f"OpenAI 클라이언트 초기화 완료")

In [ ]:
# 뉴스 데이터 로드
data_path = Path("data/news_articles.json")
with open(data_path, "r", encoding="utf-8") as f:
    articles = json.load(f)

print(f"뉴스 기사 {len(articles)}건 로드 완료")
print(f"\n첫 번째 기사 미리보기:")
print(f"  제목: {articles[0]['title']}")
print(f"  내용: {articles[0]['content'][:80]}...")
print(f"  출처: {articles[0]['source']} | 날짜: {articles[0]['date']}")

---
## 2. 프롬프트 설계 (핵심)

동일한 기사에 대해 **3가지 프롬프트 전략**을 비교합니다.

| 전략 | 설명 | 기대 품질 |
|------|------|----------|
| 기본 프롬프트 | 단순 지시 | 낮음 |
| 스키마 주입 | 엔티티/관계 타입 정의 포함 | 중간 |
| Few-shot | 입출력 예시 2개 포함 | 높음 |

### 2.1 기본 프롬프트

아무런 가이드 없이 엔티티와 관계 추출을 요청합니다.

In [ ]:
# 기본 프롬프트: 최소한의 지시
BASIC_PROMPT = """다음 텍스트에서 엔티티(개체)와 관계를 추출하세요.

결과를 다음 JSON 형식으로 반환하세요:
{
  "entities": [{"name": "엔티티명", "type": "타입"}],
  "relations": [{"source": "출발 엔티티", "relation": "관계명", "target": "도착 엔티티"}]
}

텍스트:
{text}"""

print("기본 프롬프트 정의 완료")
print(f"프롬프트 길이: {len(BASIC_PROMPT)} 문자")

### 2.2 스키마 주입 프롬프트

추출할 엔티티 타입, 관계 타입, 속성을 명시적으로 정의합니다.  
LLM이 "어떤 것을 추출해야 하는지" 정확히 알 수 있습니다.

In [ ]:
# 스키마 주입 프롬프트: 엔티티/관계 타입을 명시
SCHEMA_PROMPT = """당신은 Knowledge Graph 구축 전문가입니다.
다음 텍스트에서 엔티티와 관계를 추출하세요.

## 추출할 엔티티 타입
- Company: 기업, 회사, 법인 (속성: name, industry)
- Person: 인물, 임원, 대표 (속성: name, title)
- Product: 제품, 서비스, 모델, 기술 (속성: name, category)
- Location: 지역, 시설, 캠퍼스 (속성: name)
- Event: 투자, 출시, 협력, 개발 (속성: name, date)

## 추출할 관계 타입
- DEVELOPS: 기업 → 제품 (기업이 제품/기술을 개발)
- INVESTS_IN: 기업 → 기업/제품 (투자)
- COMPETES_WITH: 기업 ↔ 기업 (경쟁)
- PARTNERS_WITH: 기업 ↔ 기업 (협력/파트너십)
- LEADS: 인물 → 기업 (대표/임원)
- LOCATED_AT: 기업/시설 → 지역 (위치)
- SUPPLIES_TO: 기업 → 기업 (공급)
- USES: 기업 → 제품/기술 (사용/활용)

## 속성 규칙
- 엔티티 이름은 정식 명칭 사용 (예: "삼성전자", "SK하이닉스")
- 관계에 금액, 날짜 등 속성이 있으면 포함

결과를 다음 JSON 형식으로 반환하세요:
{
  "entities": [
    {"name": "엔티티명", "type": "Company|Person|Product|Location|Event", "properties": {}}
  ],
  "relations": [
    {"source": "출발 엔티티", "relation": "관계타입", "target": "도착 엔티티", "properties": {}}
  ]
}

텍스트:
{text}"""

print("스키마 주입 프롬프트 정의 완료")
print(f"프롬프트 길이: {len(SCHEMA_PROMPT)} 문자")

### 2.3 Few-shot 프롬프트

스키마 정의에 더해 **입력-출력 예시 2개**를 포함합니다.  
LLM이 원하는 추출 스타일을 정확히 학습할 수 있습니다.

In [ ]:
# Few-shot 프롬프트: 스키마 + 예시 2개
FEWSHOT_PROMPT = """당신은 Knowledge Graph 구축 전문가입니다.
다음 텍스트에서 엔티티와 관계를 추출하세요.

## 추출할 엔티티 타입
- Company: 기업, 회사, 법인 (속성: name, industry)
- Person: 인물, 임원, 대표 (속성: name, title)
- Product: 제품, 서비스, 모델, 기술 (속성: name, category)
- Location: 지역, 시설, 캠퍼스 (속성: name)
- Event: 투자, 출시, 협력, 개발 (속성: name, date)

## 추출할 관계 타입
- DEVELOPS: 기업 → 제품
- INVESTS_IN: 기업 → 기업/제품
- COMPETES_WITH: 기업 ↔ 기업
- PARTNERS_WITH: 기업 ↔ 기업
- LEADS: 인물 → 기업
- LOCATED_AT: 기업/시설 → 지역
- SUPPLIES_TO: 기업 → 기업
- USES: 기업 → 제품/기술

## 예시 1

입력: "현대모비스가 자율주행 센서를 개발한다. 정의선 회장이 이끄는 현대차그룹은 미국 보스턴에 연구소를 설립했다."

출력:
```json
{
  "entities": [
    {"name": "현대모비스", "type": "Company", "properties": {"industry": "자동차부품"}},
    {"name": "자율주행 센서", "type": "Product", "properties": {"category": "센서"}},
    {"name": "정의선", "type": "Person", "properties": {"title": "회장"}},
    {"name": "현대차그룹", "type": "Company", "properties": {"industry": "자동차"}},
    {"name": "보스턴", "type": "Location", "properties": {}}
  ],
  "relations": [
    {"source": "현대모비스", "relation": "DEVELOPS", "target": "자율주행 센서", "properties": {}},
    {"source": "정의선", "relation": "LEADS", "target": "현대차그룹", "properties": {}},
    {"source": "현대차그룹", "relation": "LOCATED_AT", "target": "보스턴", "properties": {"facility": "연구소"}}
  ]
}
```

## 예시 2

입력: "SK텔레콤이 AI 스피커 '누구'를 출시했다. SKT는 구글과 AI 기술 제휴를 맺었다."

출력:
```json
{
  "entities": [
    {"name": "SK텔레콤", "type": "Company", "properties": {"industry": "통신"}},
    {"name": "누구", "type": "Product", "properties": {"category": "AI 스피커"}},
    {"name": "구글", "type": "Company", "properties": {"industry": "IT"}}
  ],
  "relations": [
    {"source": "SK텔레콤", "relation": "DEVELOPS", "target": "누구", "properties": {}},
    {"source": "SK텔레콤", "relation": "PARTNERS_WITH", "target": "구글", "properties": {"type": "AI 기술 제휴"}}
  ]
}
```

## 규칙
- 엔티티 이름은 텍스트에 나온 정식 명칭 사용
- 같은 엔티티의 약칭이 있으면 정식 명칭으로 통일 (예: SKT → SK텔레콤)
- 관계에 금액, 날짜, 목적 등 속성이 있으면 properties에 포함
- JSON만 반환하고, 추가 설명은 하지 마세요

결과를 JSON 형식으로 반환하세요.

텍스트:
{text}"""

print("Few-shot 프롬프트 정의 완료")
print(f"프롬프트 길이: {len(FEWSHOT_PROMPT)} 문자")

### 2.4 세 가지 프롬프트 비교 실험

동일한 기사(1번)에 대해 3가지 프롬프트를 적용하고 결과를 비교합니다.

In [ ]:
def call_openai(prompt: str, text: str, model: str = "gpt-4o") -> dict:
    """OpenAI API를 호출하여 엔티티/관계를 추출합니다.
    
    Args:
        prompt: 프롬프트 템플릿 ({text} 플레이스홀더 포함)
        text: 추출 대상 텍스트
        model: 사용할 모델명
    
    Returns:
        추출된 엔티티/관계 딕셔너리
    """
    formatted_prompt = prompt.format(text=text)
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a Knowledge Graph extraction expert. Always respond in valid JSON."},
            {"role": "user", "content": formatted_prompt}
        ],
        response_format={"type": "json_object"},  # JSON 응답 강제
        temperature=0.0  # 재현성을 위해 temperature 0
    )
    
    # 토큰 사용량 기록
    usage = response.usage
    result = json.loads(response.choices[0].message.content)
    result["_usage"] = {
        "prompt_tokens": usage.prompt_tokens,
        "completion_tokens": usage.completion_tokens,
        "total_tokens": usage.total_tokens
    }
    
    return result

print("OpenAI 호출 함수 정의 완료")

In [ ]:
# 테스트 기사: 1번 (삼성전자 AI 반도체)
test_article = articles[0]
test_text = test_article["content"]

print(f"테스트 기사: {test_article['title']}")
print(f"내용: {test_text}")
print("\n" + "="*60)

# 3가지 프롬프트로 추출 실행
prompts = {
    "기본": BASIC_PROMPT,
    "스키마 주입": SCHEMA_PROMPT,
    "Few-shot": FEWSHOT_PROMPT
}

results = {}
for name, prompt in prompts.items():
    print(f"\n[{name} 프롬프트] 추출 중...")
    result = call_openai(prompt, test_text)
    results[name] = result
    
    entities = result.get("entities", [])
    relations = result.get("relations", [])
    usage = result.get("_usage", {})
    
    print(f"  엔티티: {len(entities)}개")
    for e in entities:
        print(f"    - {e['name']} ({e.get('type', 'N/A')})")
    print(f"  관계: {len(relations)}개")
    for r in relations:
        print(f"    - {r['source']} --[{r['relation']}]--> {r['target']}")
    print(f"  토큰: {usage.get('total_tokens', 'N/A')}")

In [ ]:
# 프롬프트별 비교 요약표
comparison = []
for name, result in results.items():
    entities = result.get("entities", [])
    relations = result.get("relations", [])
    usage = result.get("_usage", {})
    
    # 엔티티 타입 분포
    type_counts = {}
    for e in entities:
        t = e.get("type", "Unknown")
        type_counts[t] = type_counts.get(t, 0) + 1
    
    # 속성 포함 비율
    entities_with_props = sum(1 for e in entities if e.get("properties", {}))
    
    comparison.append({
        "프롬프트": name,
        "엔티티 수": len(entities),
        "관계 수": len(relations),
        "엔티티 타입 종류": len(type_counts),
        "속성 포함 엔티티": entities_with_props,
        "총 토큰": usage.get("total_tokens", 0),
        "입력 토큰": usage.get("prompt_tokens", 0),
        "출력 토큰": usage.get("completion_tokens", 0)
    })

df_comparison = pd.DataFrame(comparison)
print("\n프롬프트 비교 요약:")
df_comparison

### 핵심 관찰

| 프롬프트 | 장점 | 단점 |
|----------|------|------|
| 기본 | 간단, 빠름 | 타입 불일치, 누락 많음 |
| 스키마 주입 | 일관된 타입, 속성 포함 | 프롬프트 길이 증가 |
| Few-shot | 최고 품질, 정식 명칭 통일 | 토큰 비용 최대 |

**결론:** 품질이 중요한 프로덕션에서는 **Few-shot 프롬프트**를 사용합니다.

---
## 3. 자동 추출 실행

Few-shot 프롬프트를 사용하여 **10개 뉴스 기사 전체**를 배치 추출합니다.

In [ ]:
def extract_from_article(article: dict, prompt: str = FEWSHOT_PROMPT) -> dict:
    """단일 기사에서 엔티티/관계를 추출합니다.
    
    Args:
        article: 기사 딕셔너리 (id, title, content 포함)
        prompt: 사용할 프롬프트 템플릿
    
    Returns:
        추출 결과 딕셔너리 (article_id, entities, relations, usage 포함)
    """
    result = call_openai(prompt, article["content"])
    result["article_id"] = article["id"]
    result["article_title"] = article["title"]
    return result

print("추출 함수 정의 완료")

In [ ]:
# 전체 10개 기사 배치 추출
all_results = []
total_tokens = 0
total_cost = 0.0

# GPT-4o 가격 (2024년 기준, 1K 토큰당)
PRICE_INPUT = 0.0025   # $2.50 / 1M input tokens
PRICE_OUTPUT = 0.0100  # $10.00 / 1M output tokens

print("배치 추출 시작...")
print("=" * 60)

for i, article in enumerate(articles):
    print(f"\n[{i+1}/{len(articles)}] {article['title']}")
    
    start_time = time.time()
    result = extract_from_article(article)
    elapsed = time.time() - start_time
    
    # 통계 수집
    usage = result.get("_usage", {})
    input_tokens = usage.get("prompt_tokens", 0)
    output_tokens = usage.get("completion_tokens", 0)
    tokens = usage.get("total_tokens", 0)
    cost = (input_tokens * PRICE_INPUT + output_tokens * PRICE_OUTPUT) / 1000
    
    total_tokens += tokens
    total_cost += cost
    
    n_entities = len(result.get("entities", []))
    n_relations = len(result.get("relations", []))
    
    print(f"  엔티티: {n_entities}개 | 관계: {n_relations}개 | "
          f"토큰: {tokens} | 비용: ${cost:.4f} | 시간: {elapsed:.1f}초")
    
    all_results.append(result)
    
    # API Rate Limit 방지
    time.sleep(0.5)

print("\n" + "=" * 60)
print(f"배치 추출 완료!")
print(f"  총 토큰: {total_tokens:,}")
print(f"  총 비용: ${total_cost:.4f}")
print(f"  기사당 평균 비용: ${total_cost/len(articles):.4f}")

In [ ]:
# 추출 결과 저장
output_path = Path("data/llm_extraction_results.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print(f"추출 결과 저장 완료: {output_path}")

# 전체 추출 통계
total_entities = sum(len(r.get("entities", [])) for r in all_results)
total_relations = sum(len(r.get("relations", [])) for r in all_results)

print(f"\n전체 통계:")
print(f"  총 엔티티: {total_entities}개")
print(f"  총 관계: {total_relations}개")
print(f"  기사당 평균 엔티티: {total_entities/len(articles):.1f}개")
print(f"  기사당 평균 관계: {total_relations/len(articles):.1f}개")

In [ ]:
# 기사별 추출 결과 요약표
summary_rows = []
for r in all_results:
    entities = r.get("entities", [])
    relations = r.get("relations", [])
    usage = r.get("_usage", {})
    
    # 엔티티 타입 분포
    type_dist = {}
    for e in entities:
        t = e.get("type", "Unknown")
        type_dist[t] = type_dist.get(t, 0) + 1
    
    summary_rows.append({
        "기사 ID": r["article_id"],
        "제목": r["article_title"][:20] + "...",
        "엔티티": len(entities),
        "관계": len(relations),
        "Company": type_dist.get("Company", 0),
        "Person": type_dist.get("Person", 0),
        "Product": type_dist.get("Product", 0),
        "Location": type_dist.get("Location", 0),
        "토큰": usage.get("total_tokens", 0)
    })

df_summary = pd.DataFrame(summary_rows)
print("기사별 추출 결과:")
df_summary

---
## 4. 품질 비교: 수작업 vs LLM

Part 2에서 수동으로 추출한 결과와 LLM 자동 추출 결과를 비교합니다.  
(여기서는 수작업 결과를 시뮬레이션합니다 - 실제로는 Part 2 실습 결과를 사용하세요)

In [ ]:
# 수작업 추출 결과 (Part 2에서 작성한 결과를 시뮬레이션)
# 실제 실습에서는 Part 2에서 저장한 파일을 로드하세요
manual_results = {
    "article_1": {
        "entities": [
            {"name": "삼성전자", "type": "Company"},
            {"name": "경계현", "type": "Person"},
            {"name": "HBM3E", "type": "Product"},
            {"name": "SK하이닉스", "type": "Company"},
            {"name": "엔비디아", "type": "Company"},
            {"name": "평택 캠퍼스", "type": "Location"}
        ],
        "relations": [
            {"source": "삼성전자", "relation": "DEVELOPS", "target": "HBM3E"},
            {"source": "경계현", "relation": "LEADS", "target": "삼성전자"},
            {"source": "삼성전자", "relation": "COMPETES_WITH", "target": "SK하이닉스"},
            {"source": "삼성전자", "relation": "PARTNERS_WITH", "target": "엔비디아"},
            {"source": "삼성전자", "relation": "LOCATED_AT", "target": "평택 캠퍼스"}
        ]
    },
    "article_2": {
        "entities": [
            {"name": "네이버", "type": "Company"},
            {"name": "하이퍼클로바X", "type": "Product"},
            {"name": "네이버클라우드", "type": "Company"},
            {"name": "삼성SDS", "type": "Company"},
            {"name": "LG CNS", "type": "Company"},
            {"name": "최수연", "type": "Person"}
        ],
        "relations": [
            {"source": "네이버", "relation": "DEVELOPS", "target": "하이퍼클로바X"},
            {"source": "최수연", "relation": "LEADS", "target": "네이버"},
            {"source": "네이버클라우드", "relation": "SUPPLIES_TO", "target": "삼성SDS"},
            {"source": "네이버클라우드", "relation": "SUPPLIES_TO", "target": "LG CNS"}
        ]
    }
}

print("수작업 추출 결과 로드 완료 (시뮬레이션)")
print(f"  기사 수: {len(manual_results)}")
for key, val in manual_results.items():
    print(f"  {key}: 엔티티 {len(val['entities'])}개, 관계 {len(val['relations'])}개")

In [ ]:
def compare_extraction(manual: dict, llm_result: dict, article_id: int) -> dict:
    """수작업과 LLM 추출 결과를 비교합니다.
    
    Args:
        manual: 수작업 추출 결과
        llm_result: LLM 추출 결과
        article_id: 기사 ID
    
    Returns:
        비교 메트릭 딕셔너리
    """
    # 수작업 엔티티/관계
    manual_entities = {e["name"] for e in manual["entities"]}
    manual_relations = {
        (r["source"], r["relation"], r["target"]) 
        for r in manual["relations"]
    }
    
    # LLM 엔티티/관계
    llm_entities = {e["name"] for e in llm_result.get("entities", [])}
    llm_relations = {
        (r["source"], r["relation"], r["target"]) 
        for r in llm_result.get("relations", [])
    }
    
    # 엔티티 비교
    entity_overlap = manual_entities & llm_entities
    entity_only_manual = manual_entities - llm_entities
    entity_only_llm = llm_entities - manual_entities
    
    # 관계 비교
    relation_overlap = manual_relations & llm_relations
    relation_only_manual = manual_relations - llm_relations
    relation_only_llm = llm_relations - manual_relations
    
    # 정밀도/재현율 계산
    entity_precision = len(entity_overlap) / len(llm_entities) if llm_entities else 0
    entity_recall = len(entity_overlap) / len(manual_entities) if manual_entities else 0
    entity_f1 = (2 * entity_precision * entity_recall / 
                 (entity_precision + entity_recall)) if (entity_precision + entity_recall) > 0 else 0
    
    return {
        "기사 ID": article_id,
        "수작업 엔티티": len(manual_entities),
        "LLM 엔티티": len(llm_entities),
        "공통 엔티티": len(entity_overlap),
        "수작업만": len(entity_only_manual),
        "LLM만": len(entity_only_llm),
        "수작업 관계": len(manual_relations),
        "LLM 관계": len(llm_relations),
        "공통 관계": len(relation_overlap),
        "엔티티 정밀도": round(entity_precision, 3),
        "엔티티 재현율": round(entity_recall, 3),
        "엔티티 F1": round(entity_f1, 3),
        "수작업만 엔티티 목록": entity_only_manual,
        "LLM만 엔티티 목록": entity_only_llm
    }

print("비교 함수 정의 완료")

In [ ]:
# 기사 1, 2에 대해 수작업 vs LLM 비교
comparisons = []

for article_key, manual in manual_results.items():
    article_id = int(article_key.split("_")[1])
    
    # 해당 기사의 LLM 추출 결과 찾기
    llm_result = next(
        (r for r in all_results if r["article_id"] == article_id), 
        None
    )
    
    if llm_result:
        comp = compare_extraction(manual, llm_result, article_id)
        comparisons.append(comp)
        
        print(f"\n기사 {article_id} 비교 결과:")
        print(f"  엔티티 - 수작업: {comp['수작업 엔티티']}개 | LLM: {comp['LLM 엔티티']}개 | 공통: {comp['공통 엔티티']}개")
        print(f"  관계   - 수작업: {comp['수작업 관계']}개 | LLM: {comp['LLM 관계']}개 | 공통: {comp['공통 관계']}개")
        print(f"  엔티티 F1: {comp['엔티티 F1']}")
        
        if comp['수작업만 엔티티 목록']:
            print(f"  수작업에만 있는 엔티티: {comp['수작업만 엔티티 목록']}")
        if comp['LLM만 엔티티 목록']:
            print(f"  LLM에만 있는 엔티티: {comp['LLM만 엔티티 목록']}")

In [ ]:
# 비교 시각화: 바 차트
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 비교 데이터 준비
article_ids = [c["기사 ID"] for c in comparisons]
x = range(len(article_ids))
width = 0.25

# 엔티티 수 비교
ax1 = axes[0]
manual_entities = [c["수작업 엔티티"] for c in comparisons]
llm_entities = [c["LLM 엔티티"] for c in comparisons]
common_entities = [c["공통 엔티티"] for c in comparisons]

bars1 = ax1.bar([i - width for i in x], manual_entities, width, label="수작업", color="#3b82f6")
bars2 = ax1.bar([i for i in x], llm_entities, width, label="LLM", color="#8b5cf6")
bars3 = ax1.bar([i + width for i in x], common_entities, width, label="공통", color="#0ea5e9")

ax1.set_xlabel("기사 ID")
ax1.set_ylabel("엔티티 수")
ax1.set_title("엔티티 추출 비교: 수작업 vs LLM")
ax1.set_xticks(x)
ax1.set_xticklabels([f"기사 {id}" for id in article_ids])
ax1.legend()

# 관계 수 비교
ax2 = axes[1]
manual_relations = [c["수작업 관계"] for c in comparisons]
llm_relations = [c["LLM 관계"] for c in comparisons]
common_relations = [c["공통 관계"] for c in comparisons]

bars4 = ax2.bar([i - width for i in x], manual_relations, width, label="수작업", color="#3b82f6")
bars5 = ax2.bar([i for i in x], llm_relations, width, label="LLM", color="#8b5cf6")
bars6 = ax2.bar([i + width for i in x], common_relations, width, label="공통", color="#0ea5e9")

ax2.set_xlabel("기사 ID")
ax2.set_ylabel("관계 수")
ax2.set_title("관계 추출 비교: 수작업 vs LLM")
ax2.set_xticks(x)
ax2.set_xticklabels([f"기사 {id}" for id in article_ids])
ax2.legend()

plt.tight_layout()
plt.show()

print("\n핵심 인사이트:")
print("- LLM은 일반적으로 수작업보다 더 많은 엔티티를 추출 (높은 재현율)")
print("- 수작업은 정밀도가 높지만, 누락이 발생할 수 있음")
print("- LLM 추출은 일관성이 높고, 대량 처리에 적합")

---
## 5. Neo4j 배치 적재

LLM 추출 결과를 Neo4j에 자동으로 적재합니다.  
Cypher MERGE 문을 사용하여 중복 없이 노드와 관계를 생성합니다.

In [ ]:
def build_merge_queries(extraction_result: dict) -> list[str]:
    """추출 결과를 Cypher MERGE 문으로 변환합니다.
    
    Args:
        extraction_result: LLM 추출 결과 딕셔너리
    
    Returns:
        Cypher 쿼리 문자열 리스트
    """
    queries = []
    
    # 엔티티 노드 생성
    for entity in extraction_result.get("entities", []):
        name = entity["name"].replace('"', '\\"')
        entity_type = entity.get("type", "Entity")
        props = entity.get("properties", {})
        
        # 속성 문자열 생성
        prop_parts = [f'name: "{name}"']
        for k, v in props.items():
            if isinstance(v, str):
                prop_parts.append(f'{k}: "{v}"')
            else:
                prop_parts.append(f'{k}: {v}')
        prop_str = ", ".join(prop_parts)
        
        query = f'MERGE (n:{entity_type} {{name: "{name}"}}) SET n += {{{prop_str}}}'
        queries.append(query)
    
    # 관계 생성
    for rel in extraction_result.get("relations", []):
        source = rel["source"].replace('"', '\\"')
        target = rel["target"].replace('"', '\\"')
        rel_type = rel["relation"]
        props = rel.get("properties", {})
        
        # 관계 속성
        if props:
            prop_parts = []
            for k, v in props.items():
                if isinstance(v, str):
                    prop_parts.append(f'{k}: "{v}"')
                else:
                    prop_parts.append(f'{k}: {v}')
            prop_str = " {" + ", ".join(prop_parts) + "}"
        else:
            prop_str = ""
        
        query = (
            f'MATCH (a {{name: "{source}"}}) '
            f'MATCH (b {{name: "{target}"}}) '
            f'MERGE (a)-[r:{rel_type}]->(b)'
        )
        if props:
            query += f' SET r += {{{";".join(prop_parts).replace(";", ", ")}}}'
        
        queries.append(query)
    
    return queries

# 테스트: 첫 번째 기사의 쿼리 생성
test_queries = build_merge_queries(all_results[0])
print(f"생성된 Cypher 쿼리 수: {len(test_queries)}")
print("\n처음 3개 쿼리 미리보기:")
for q in test_queries[:3]:
    print(f"  {q}")

In [ ]:
def load_to_neo4j(driver, extraction_results: list[dict], clear_first: bool = True):
    """추출 결과를 Neo4j에 배치 적재합니다.
    
    Args:
        driver: Neo4j 드라이버
        extraction_results: 전체 추출 결과 리스트
        clear_first: True이면 기존 데이터 삭제 후 적재
    """
    with driver.session() as session:
        # 기존 데이터 삭제 (선택)
        if clear_first:
            session.run("MATCH (n) DETACH DELETE n")
            print("기존 데이터 삭제 완료")
        
        total_queries = 0
        errors = []
        
        for result in extraction_results:
            queries = build_merge_queries(result)
            article_id = result.get("article_id", "?")
            
            # 트랜잭션으로 실행
            for query in queries:
                try:
                    session.run(query)
                    total_queries += 1
                except Exception as e:
                    errors.append({"article_id": article_id, "query": query, "error": str(e)})
        
        print(f"\n적재 완료!")
        print(f"  실행된 쿼리: {total_queries}개")
        print(f"  에러: {len(errors)}개")
        
        if errors:
            print("\n에러 목록:")
            for err in errors[:5]:
                print(f"  기사 {err['article_id']}: {err['error'][:80]}")
        
        return total_queries, errors

# Neo4j에 적재 실행
n_queries, load_errors = load_to_neo4j(driver, all_results, clear_first=True)

In [ ]:
# 적재 결과 검증
with driver.session() as session:
    # 노드 수 확인
    node_count = session.run("MATCH (n) RETURN count(n) AS cnt").single()["cnt"]
    
    # 관계 수 확인
    rel_count = session.run("MATCH ()-[r]->() RETURN count(r) AS cnt").single()["cnt"]
    
    # 노드 타입별 분포
    label_dist = session.run("""
        MATCH (n) 
        UNWIND labels(n) AS label 
        RETURN label, count(*) AS cnt 
        ORDER BY cnt DESC
    """).data()
    
    # 관계 타입별 분포
    rel_dist = session.run("""
        MATCH ()-[r]->() 
        RETURN type(r) AS relType, count(*) AS cnt 
        ORDER BY cnt DESC
    """).data()
    
    # 샘플 서브그래프
    sample = session.run("""
        MATCH (a)-[r]->(b) 
        RETURN a.name AS source, type(r) AS relation, b.name AS target 
        LIMIT 10
    """).data()

print(f"Neo4j 적재 검증:")
print(f"  총 노드: {node_count}개")
print(f"  총 관계: {rel_count}개")

print(f"\n노드 라벨 분포:")
for item in label_dist:
    print(f"  {item['label']}: {item['cnt']}개")

print(f"\n관계 타입 분포:")
for item in rel_dist:
    print(f"  {item['relType']}: {item['cnt']}개")

print(f"\n샘플 트리플 (상위 10개):")
for s in sample:
    print(f"  {s['source']} --[{s['relation']}]--> {s['target']}")

---
## 6. 연습 문제

### 연습 6.1: 프롬프트 변형 실험

Few-shot 프롬프트를 수정하여 다음을 실험해보세요:
- 예시를 3개로 늘렸을 때 품질이 향상되는지?
- system 메시지에 도메인 전문가 역할을 부여하면?
- temperature를 0.3으로 올리면 결과가 달라지는지?

In [ ]:
# 연습 6.1: 프롬프트 변형 실험
# TODO: 아래 코드를 수정하여 실험해보세요

# 실험 1: 예시 3개 Few-shot
FEWSHOT_3_PROMPT = """당신은 한국 IT 산업 전문 Knowledge Graph 구축가입니다.

다음 텍스트에서 엔티티와 관계를 추출하세요.
스키마와 예시는 앞서 정의한 것과 동일합니다.

# TODO: 여기에 예시 3개를 추가하세요
# 예시 1: ...
# 예시 2: ...
# 예시 3: ...

텍스트:
{text}"""

# 실험 2: temperature 비교
# TODO: call_openai 함수를 수정하여 temperature 파라미터를 추가하고
# temperature=0.0 vs 0.3 vs 0.7 결과를 비교해보세요

print("연습 6.1: 코드를 수정하여 실험해보세요!")

### 연습 6.2: 제조 도메인 문서 추출

IT 뉴스 대신 **제조 도메인 문서**에서 엔티티/관계를 추출해보세요.  
도메인이 다르면 스키마를 어떻게 바꿔야 할까요?

In [ ]:
# 연습 6.2: 제조 도메인 문서 추출

# 제조 도메인 데이터 로드
mfg_path = Path("data/manufacturing_docs.json")
with open(mfg_path, "r", encoding="utf-8") as f:
    mfg_docs = json.load(f)

print(f"제조 도메인 문서 {len(mfg_docs)}건 로드")
for doc in mfg_docs:
    print(f"  - {doc['title']}")

# TODO: 제조 도메인에 맞는 스키마를 정의하세요
# 힌트: 엔티티 타입 - Equipment, Material, Process, Defect, Person
#        관계 타입 - PRODUCES, USES_MATERIAL, CAUSES_DEFECT, MAINTAINED_BY, SUPPLIES

MFG_SCHEMA_PROMPT = """당신은 제조업 Knowledge Graph 구축 전문가입니다.

## 추출할 엔티티 타입
- Equipment: 설비, 장비 (속성: name, model, specs)
- Material: 원자재, 부품 (속성: name, type)
- Process: 공정, 작업 (속성: name, conditions)
- Defect: 불량, 결함 (속성: name, rate)
- Person: 담당자 (속성: name, role)
- Company: 공급업체 (속성: name)

## 추출할 관계 타입
- PERFORMS: 설비 → 공정
- USES_MATERIAL: 공정 → 원자재
- CAUSES: 설비/공정 → 불량
- MAINTAINED_BY: 설비 → 담당자
- SUPPLIES: 업체 → 원자재/설비
- NEXT_STEP: 공정 → 공정 (순서)

텍스트에서 엔티티와 관계를 추출하세요. JSON 형식으로 반환하세요.

텍스트:
{text}"""

# 첫 번째 제조 문서로 테스트
mfg_result = call_openai(MFG_SCHEMA_PROMPT, mfg_docs[0]["content"])

print(f"\n제조 문서 추출 결과:")
print(f"  엔티티: {len(mfg_result.get('entities', []))}개")
for e in mfg_result.get("entities", []):
    print(f"    - {e['name']} ({e.get('type', 'N/A')})")
print(f"  관계: {len(mfg_result.get('relations', []))}개")
for r in mfg_result.get("relations", []):
    print(f"    - {r['source']} --[{r['relation']}]--> {r['target']}")

---
## 정리

### 이번 파트에서 배운 것

1. **프롬프트 설계가 품질을 결정한다** - 기본 < 스키마 주입 < Few-shot 순으로 품질 향상
2. **JSON 응답 강제** (`response_format`) 로 파싱 오류를 방지할 수 있다
3. **LLM은 재현율이 높고**, 수작업은 정밀도가 높은 경향이 있다
4. **배치 처리 시 토큰 비용 추적**이 중요하다
5. **도메인별 스키마 커스터마이징**이 필수적이다

### 다음 파트 예고

**Part 4: Entity Resolution** - LLM이 추출한 "삼성전자", "Samsung", "삼성" 같은 중복 엔티티를 통합합니다.

In [ ]:
# 리소스 정리
driver.close()
print("Neo4j 연결 종료")
print("Part 3 실습 완료!")